In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import seaborn as sns
import sklearn
import imblearn
import matplotlib.pyplot as plt
import time
import sklearn.metrics as m
import warnings

warnings.filterwarnings('ignore')


In [8]:
#Load Data

In [7]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

#Probably can`t be finished because of huge amount of data with kaggle hardware, add nrows parameter to run here

In [6]:
#Setting

In [12]:
df1=pd.read_csv("./archive/Friday-WorkingHours-Afternoon-DDos.csv")#,nrows = 50000
df2=pd.read_csv("./archive/Friday-WorkingHours-Afternoon-PortScan.csv")
df3=pd.read_csv("./archive/Friday-WorkingHours-Morning.csv")
df4=pd.read_csv("./archive/Monday-WorkingHours.csv")
df5=pd.read_csv("./archive/Thursday-WorkingHours-Afternoon-Infilteration.csv")
df6=pd.read_csv("./archive/Thursday-WorkingHours-Morning-WebAttacks.csv")
df7=pd.read_csv("./archive/Tuesday-WorkingHours.csv")
df8=pd.read_csv("./archive/Wednesday-workingHours.csv")

In [14]:
import xgboost as xgb

df = pd.concat([df1,df2])
del df1,df2
df = pd.concat([df,df3])
del df3
df = pd.concat([df,df4])
del df4
df = pd.concat([df,df5])
del df5
df = pd.concat([df,df6])
del df6
df = pd.concat([df,df7])
del df7
df = pd.concat([df,df8])
del df8

print(df.info())
print(df.head())

NameError: name 'df1' is not defined

In [ ]:
for i in df.coulms:
    df  = df[df[i]!="Infinity"]
    df  = df[df[i]!= np.nan]
    df = df[df[[i]!= ", ,"]
    df[['Flow Byte/s', ' Flow Packets/s']] = df[['Flow Byte/s', ' Flow Packets/s']].apply(pd.to_numeric)

In [ ]:
print(df[' Bwd PSH Flags'].value_counts())
print(df[' Bwd URG Flags'].value_counts())
print(df['Fwd Avg Bytes/Bulk'].value_counts())
print(df[' Fwd Avg Packets/Bulk'].value_counts())
print(df[' Fwd Avg Bulk Rate'].value_counts())
print(df[' Bwd Avg Bytes/Bulk'].value_counts())
print(df[' Bwd Avg Packets/Bulk'].value_counts())
print(df['Bwd Avg Bulk Rate'].value_counts())
    
#Date preprocesing
df.drop([' Bwd PSH Flags'], axis=1, inplace=True)
df.drop([' Bwd URG Flags'], axis=1, inplace=True)
df.drop(['Fwd Avg Bytes/Bulk'], axis=1, inplace=True)
df.drop([' Fwd Avg Packets/Bulk'], axis=1, inplace=True)
df.drop([' Fwd Avg Bulk Rate'], axis=1, inplace=True)
df.drop([' Bwd Avg Bytes/Bulk'], axis=1, inplace=True)
df.drop([' Bwd Avg Packets/Bulk'], axis=1, inplace=True)
df.drop(['Bwd Avg Bulk Rate'], axis=1, inplace=True)

df.info()
df.head()

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df,test_size=0.3,random_state=10)

train.describe()
test.describe()

train[' Label'].value_counts()
test[' Label'].value_counts()


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

#숫자 속성과 스케일을 제로민과 유닛 분산을 취한다.
cols = train.select_dtypes(include=['float64','int64']).colums
sc_train = scaler.fit_transform(train.selct_dtypes(include=['float64','int64']))
sc_test = scaler.fit_transform(test.selct_dtypes(include=['float64','int64']))

# 그결과를 데이터 프레임으로 리턴
sc_traindf = pd.DataFrame(sc_train,colums = cols)
sc_test = pd.DataFrame(sc_test,colums = cols)


In [ ]:
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder()

trainDep = train[' Label'].value.reshape(-1,1)
trainDep = onehotencoder.fit_transform(trainDep).toarray()
testDep = test[' Label'].value.reshape(-1,1)
testDep = onehotencoder.fit_transform(testDep).toarray()
